In [139]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
import re

import mp_run

from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error

import os

from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from scipy import stats

from multiprocessing import Pool, cpu_count


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# ts training/testing data curation 

df_1 = pd.read_csv('../data/yeast/GSE145936_Sis1-AA_Gene_counts_normalized.txt', sep='\t', index_col=0)
df_2 = pd.read_csv('../data/yeast/GSE153609_gene_expression_TPM_all_times.csv', index_col=0)
df_3 = pd.read_csv('../data/yeast/GSE168699_RNA_TPM_all_times.csv', index_col=0)

to_drop = df_3.columns[:7]
df_3 = df_3.drop(labels=to_drop, axis=1)
df_1 = df_1.drop(labels=['gene name'], axis=1)

common_genes = set(df_1.index).intersection(set(df_2.index)).intersection(set(df_3.index))
common_genes = list(common_genes)
df_1 = df_1.loc[common_genes]
df_2 = df_2.loc[common_genes]
df_3 = df_3.loc[common_genes]



In [75]:

# normalized_df_1=(df_1-df_1.min())/(df_1.max()-df_1.min())
# normalized_df_2=(df_2-df_2.min())/(df_2.max()-df_2.min())
# normalized_df_3=(df_3-df_3.min())/(df_3.max()-df_3.min())
normalized_df_1=df_1.apply(stats.zscore, axis=0)
normalized_df_2=df_2.apply(stats.zscore, axis=0)
normalized_df_3=df_3.apply(stats.zscore, axis=0)
# normalized_df_1 = normalized_df_1*100.0
# normalized_df_2 = normalized_df_2*100.0
# normalized_df_3 = normalized_df_3*100.0

test_df_1 = normalized_df_1.iloc[:,[3,4,5,9,10,11]]
test_df_2 = normalized_df_2.iloc[:,[3,4,5]]
test_df_3 = normalized_df_3.iloc[:, -5:]
test_exp = pd.concat([test_df_1, test_df_2, test_df_3], axis=1)
test_source = test_exp.iloc[:,[0,1,3,4,6,7,9,10,11,12]]
test_target = test_exp.iloc[:,[0,1,3,4,6,7,9,10,11,12]]

train_source_df_1 = normalized_df_1.iloc[:, [0,1,2,3,5,6,7,8,9]]
train_target_df_1 = normalized_df_1.iloc[:, [1,2,3,4,6,7,8,9,10]]
train_source_df_2 = normalized_df_2.iloc[:, [0,1,2,3]]
train_target_df_2 = normalized_df_2.iloc[:, [1,2,3,4]]
train_source_df_3 = normalized_df_3.iloc[:, :-4]
train_target_df_3 = normalized_df_3.iloc[:, 1:-3]
train_source = pd.concat([train_source_df_1, train_source_df_2, train_source_df_3], axis=1)
train_target = pd.concat([train_target_df_1, train_target_df_2, train_target_df_3], axis=1)
source_exp = pd.concat([train_source, test_source], axis=1)
target_exp = pd.concat([train_target, test_target], axis=1)

In [129]:
high_var_target_genes = set(normalized_df_1.var(axis=1).sort_values(ascending=False).head(500).index).intersection(
    set(normalized_df_2.var(axis=1).sort_values(ascending=False).head(500).index)).intersection(
    set(normalized_df_3.var(axis=1).sort_values(ascending=False).head(500).index))
high_var_target_genes = high_var_target_genes.difference(set(tf_list))

high_var_target_genes = list(high_var_target_genes)

In [76]:
# get network data, training features
network_df = pd.read_csv('./yeat_network.csv', index_col=0)
tf_set = set()
target_gene_list = []
for i, row in network_df.iterrows():
    tf_list = row.tf_list
    if pd.isnull(tf_list): 
        continue
    tf_list = tf_list.split('; ')
    tf_set = tf_set.union(set(tf_list))
    target_gene_list.append(i)

target_gene_list = list(set(target_gene_list).intersection(set(common_genes)))
# filterout NaN target:
target_gene_list = list(target_exp.loc[target_gene_list][target_exp.loc[target_gene_list].isnull().any(axis=1)==False].index)
tf_list = list(tf_set.intersection(set(common_genes)))

X = source_exp.loc[tf_list]


In [92]:
len(set(target_gene_list).difference(set(tf_list)))

4684

In [77]:
mp_calc = mp_run.MpCalc(target_gene_list, target_exp, X, network_df, train_source.loc[tf_list], train_target, test_source.loc[tf_list], test_target)

In [78]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = np.array(list(tqdm(p.imap(mp_calc.network_v_model, range(iter_length)), total=iter_length)))

100%|████████████████████████████████████████████████████████████████████████████| 4897/4897 [04:47<00:00, 17.05it/s]


In [79]:
out_df = pd.DataFrame(index=target_gene_list)
out_df['best_network_score'] = r[:, 0]
out_df['mean_network_score'] = r[:, 1]
out_df['model_score'] = r[:, 2]
out_df.to_csv('./yeast_rf_model_network_res.csv')

In [81]:
out_df.sort_values('best_network_score', ascending=False)

,best_network_score,mean_network_score,model_score
YLR433C,0.784047,-0.792224,0.747670
YIR018W,0.763446,-0.834101,0.434182
YLL013C,0.761291,-0.656174,0.359910
YML087C,0.743976,-1.439675,0.767117
YGL035C,0.741165,-0.877434,0.351822
...,...,...,...
YDL184C,-225.986538,-901.078338,-149.054799
YJR010W,-262.122016,-1971.365783,-346.085762
YDR258C,-334.440433,-2205.220258,-308.344198
YGR281W,-636.120168,-2464.676030,-281.532120


In [5]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.network_all_v_model, range(iter_length)), total=iter_length))

100%|██████████| 4897/4897 [03:50<00:00, 21.25it/s]


In [6]:
r = np.array(r)
out_df = pd.DataFrame(index=target_gene_list)
out_df['best_network_score'] = r[:, 0]
out_df['model_score'] = r[:, 1]

In [11]:
out_df.sort_values('best_network_score', ascending=False).head(30)

,best_network_score,model_score
YCR096C,1.000000,1.000000
YOR059C,0.797898,0.255164
YGL062W,0.792003,0.719769
YOL158C,0.790346,-2.299205
YLR387C,0.785474,0.241728
YBL051C,0.772370,0.054555
YBR276C,0.758327,0.472882
YPL149W,0.752984,-0.868749
YCR093W,0.732183,0.217571
YIL159W,0.728257,0.582680


In [175]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.lime_test, range(iter_length)), total=iter_length))

100%|████████████████████████████████████████████████████████████████████████████| 4897/4897 [01:26<00:00, 56.54it/s]


In [176]:
r = np.array(r)
out_df = pd.DataFrame(index=target_gene_list)
out_df['network_score'] = r[:, 0]
out_df['model_score'] = r[:, 1]
out_df['moded_network_score'] = r[:, 2]
out_df['diff'] = r[:, 1] - r[:, 0]
out_df['network_model_score'] = r[:, 3]

In [174]:
out_df

,network_score,model_score,moded_network_score,diff,network_model_score
YHR021C,0.875623,-0.137734,0.741408,-1.013357,0.872521
YHR036W,0.984607,-0.163658,0.973580,-1.148265,0.973006
YJR006W,0.396579,-80.703235,-0.028012,-81.099814,0.647123
YLR043C,0.880287,0.790461,0.878052,-0.089826,0.853424
YPL169C,0.687141,-2.166235,0.381813,-2.853376,0.574200
...,...,...,...,...,...
YBR122C,-0.286955,-6.238150,-1.387137,-5.951195,-0.249628
YLR163C,0.542412,-1.073966,-0.179278,-1.616378,0.562170
YHR085W,0.841032,0.863481,0.561500,0.022450,0.891368
YMR013C,0.435212,-0.473235,0.309319,-0.908447,-0.169613


In [148]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.full_comp, range(iter_length)), total=iter_length))

100%|████████████████████████████████████████████████████████████████████████████| 4897/4897 [01:39<00:00, 49.27it/s]


In [149]:
r = np.array(r)
out_df = pd.DataFrame(index=target_gene_list)
out_df['rf_score'] = r[:, 0]
out_df['linear_score'] = r[:, 1]
out_df['gs_rf_score'] = r[:, 2]
out_df['gs_linear_score'] = r[:, 3]
out_df['rf_with_linear_top_features_score'] = r[:, 4]
out_df['linear_with_rf_top_features_score'] = r[:, 5]


In [171]:
(out_df['rf_score'] > out_df['gs_linear_score']).sum()

4468

In [152]:
out_df

,rf_score,linear_score,gs_rf_score,gs_linear_score,rf_with_linear_top_features_score,linear_with_rf_top_features_score
YHR021C,0.849148,-0.006986,0.873513,-0.137734,0.906161,0.603621
YHR036W,0.992807,0.918968,0.985085,-0.163658,0.986237,0.925525
YJR006W,0.684067,-0.469068,0.758521,-80.703235,0.474702,0.607002
YLR043C,0.865905,0.820743,0.767780,0.790461,0.866771,0.753353
YPL169C,0.650097,0.095876,0.651211,-2.166235,0.632594,0.687194
...,...,...,...,...,...,...
YBR122C,-0.251323,-0.634307,-0.027273,-6.238150,-0.005806,-2.353202
YLR163C,0.653905,0.090145,0.627649,-1.073966,0.466820,0.195506
YHR085W,0.882376,0.884564,0.895587,0.863481,0.903367,0.664933
YMR013C,0.388202,-0.448900,0.396665,-0.473235,0.096914,-0.205592


In [166]:
from itertools import combinations
model_combs = list(combinations(out_df.columns, 2))

In [169]:
for a, b in model_combs:
    t, p = stats.ttest_rel(out_df[a], out_df[b])
    if (p > 0.05):
        print('{} and {} don\'t have statistically different performance'.format(a, b))
        continue
    if (t > 0):
        print('{} has statisically better performance than {}'.format(a, b))
    else:
        
        print('{} has statisically better performance than {}'.format(b, a))

rf_score has statisically better performance than linear_score
rf_score has statisically better performance than gs_rf_score
rf_score has statisically better performance than gs_linear_score
rf_score has statisically better performance than rf_with_linear_top_features_score
rf_score has statisically better performance than linear_with_rf_top_features_score
gs_rf_score has statisically better performance than linear_score
linear_score has statisically better performance than gs_linear_score
rf_with_linear_top_features_score has statisically better performance than linear_score
linear_score has statisically better performance than linear_with_rf_top_features_score
gs_rf_score has statisically better performance than gs_linear_score
gs_rf_score has statisically better performance than rf_with_linear_top_features_score
gs_rf_score has statisically better performance than linear_with_rf_top_features_score
rf_with_linear_top_features_score has statisically better performance than gs_linear_s

In [170]:
for a, b in model_combs:
    t, p = stats.ttest_rel(out_df.loc[high_var_target_genes][a], out_df.loc[high_var_target_genes][b])
    if (p > 0.05):
        print('{} and {} don\'t have statistically different performance'.format(a, b))
        continue
    if (t > 0):
        print('{} has statisically better performance than {}'.format(a, b))
    else:
        
        print('{} has statisically better performance than {}'.format(b, a))

rf_score has statisically better performance than linear_score
rf_score has statisically better performance than gs_rf_score
rf_score has statisically better performance than gs_linear_score
rf_score has statisically better performance than rf_with_linear_top_features_score
rf_score has statisically better performance than linear_with_rf_top_features_score
gs_rf_score has statisically better performance than linear_score
linear_score and gs_linear_score don't have statistically different performance
rf_with_linear_top_features_score has statisically better performance than linear_score
linear_score and linear_with_rf_top_features_score don't have statistically different performance
gs_rf_score has statisically better performance than gs_linear_score
gs_rf_score and rf_with_linear_top_features_score don't have statistically different performance
gs_rf_score has statisically better performance than linear_with_rf_top_features_score
rf_with_linear_top_features_score has statisically bette